# AI LAB WEEK-2 Divansh Prasad (210968140)

# Exercises 
# QUESTION 1: The Game :
According to the “Six Degrees of Kevin Bacon” game, anyone in the Hollywood film industry can be\
connected to Kevin Bacon within six steps, where each step consists of finding a film that two actors\
both starred in. To solve the problem, find the shortest path between any two actors by choosing a\
sequence of movies that connects them. For example, the shortest path between Jennifer Lawrence\
and Tom Hanks is 2: Jennifer Lawrence is connected to Kevin Bacon by both starring in “X-Men: First\
Class,” and Kevin Bacon is connected to Tom Hanks by both starring in “Apollo 13.”

Problem Solving Agent:\
Given two actors nodes in the graph we need to find the distance (shortest path) between the\
nodes. Write a python program to determine how many “degrees of separation” apart two actors\
are. Find the distance or the degree of separation, using
1. Breadth first search
2. Depth first search

Distribution Code: Data & Download the distribution code from:\
https://cdn.cs50.net/ai/2020/x/projects/0/degrees.zip\
The distribution code contains two sets of CSV data files: one set in the large directory and one set in
the small directory. Use the small dataset for ease of testing and experimentation. Each dataset
consists of three CSV files.
1. small/people.csv: each person has a unique id, corresponding with their id in IMDb’s database.
They also have a name, and a birth year.
2. small/movies.csv: You’ll see here that each movie also has a unique id, in addition to a title and the
year in which the movie was released.
3. small/stars.csv: This file establishes a relationship between the people in people.csv and the movies
in movies.csv. Each row is a pair of a person_id value and movie_id value. For example: The first row
(ignoring the header), for example, states that the person with id 102 starred in the movie with id
104257. Checking that against people.csv and movies.csv, you’ll find that this line is saying that Kevin
Bacon starred in the movie “A Few Good Men.”
4. degrees.py: At the top, several data structures are defined to store information from the CSV files.
The names dictionary is a way to look up a person by their name: it maps names to a set of
corresponding ids (because it’s possible that multiple actors have the same name). The people
dictionary maps each person’s id to another dictionary with values for the person’s name, birth year,
and the set of all the movies they have starred in. And the movies dictionary maps each movie’s id to
another dictionary with values for that movie’s title, release year, and the set of all the movie’s stars.
The load_data function loads data from the CSV files into these data structures. The main function in
this program first loads data into memory (the directory from which the data is loaded can be specified
by a command-line argument). Then, the function prompts the user to type in two names. The
person_id_for_name function retrieves the id for any person (and handles prompting the user to
clarify, in the event that multiple people have the same name). The function then calls the
shortest_path function to compute the shortest path between the two people, and prints out the
path.\

Code the shortest_path function using\
1. Breadth First Search
2. Depth First Search

### Util.py

In [1]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action


class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node


class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

### Degrees.py (Without shortest path)

In [23]:
import csv
import sys

# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}


def load_data():
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]


def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors

### Shortest path function

In [27]:
def shortest_path_BFS(source, target):
    # Initialize a starting node with the source person_id
    start = Node(state=source, parent=None, action=None)

    # Initialize a frontier using QueueFrontier for BFS
    frontier = QueueFrontier()
    frontier.add(start)

    # Initialize an empty set to keep track of explored nodes
    explored = set()

    # Loop until the frontier is empty
    while not frontier.empty():
        # Remove a node from the frontier
        node = frontier.remove()

        # If the node's state is the target, reconstruct the path and return it
        if node.state == target:
            path = []
            while node.parent is not None:
                path.append((node.action, node.state))
                node = node.parent
            path.reverse()
            return path

        # Mark the node as explored
        explored.add(node.state)

        # Add neighbors to the frontier
        for action, neighbor in neighbors_for_person(node.state):
            if neighbor not in explored and not frontier.contains_state(neighbor):
                child = Node(state=neighbor, parent=node, action=action)
                frontier.add(child)

    # If no path is found, return None
    return None


def shortest_path_DFS(source, target):
    # Initialize a starting node with the source person_id
    start = Node(state=source, parent=None, action=None)

    # Initialize a frontier using StackFrontier for DFS
    frontier = StackFrontier()
    frontier.add(start)

    # Initialize an empty set to keep track of explored nodes
    explored = set()

    # Loop until the frontier is empty
    while not frontier.empty():
        # Remove a node from the frontier
        node = frontier.remove()

        # If the node's state is the target, reconstruct the path and return it
        if node.state == target:
            path = []
            while node.parent is not None:
                path.append((node.action, node.state))
                node = node.parent
            path.reverse()
            return path

        # Mark the node as explored
        explored.add(node.state)

        # Add neighbors to the frontier
        for action, neighbor in neighbors_for_person(node.state):
            if neighbor not in explored and not frontier.contains_state(neighbor):
                child = Node(state=neighbor, parent=node, action=action)
                frontier.add(child)

    # If no path is found, return None
    return None

### Input and Output

In [30]:
load_data()
source = person_id_for_name(input("Name: "))
if source is None:
    sys.exit("Person not found.")
target = person_id_for_name(input("Name: "))
if target is None:
    sys.exit("Person not found.")
b_d=int((input("Press 1 for BFS.\nPress 2 for DFS: ")))
if (b_d==1):
    path = shortest_path_BFS(source, target)
elif (b_d==2):
    path = shortest_path_DFS(source, target)
else:
    print("Invalid input.")

print()
if path is None:
    print("Not connected.")
else:
    degrees = len(path)
    print(f"{degrees} degree(s) of separation.")
    path = [(None, source)] + path
    for i in range(degrees):
        person1 = people[path[i][1]]["name"]
        person2 = people[path[i + 1][1]]["name"]
        movie = movies[path[i + 1][0]]["title"]
        print(f"{i + 1}: {person1} and {person2} starred in {movie}")

Name: Kevin Bacon
Name: Chris Sarandon
Press 1 for BFS.
Press 2 for DFS: 1

3 degree(s) of separation.
1: Kevin Bacon and Tom Hanks starred in Apollo 13
2: Tom Hanks and Robin Wright starred in Forrest Gump
3: Robin Wright and Chris Sarandon starred in The Princess Bride
